In [ ]:
import torch
import torch.nn as nn
import numpy as np


# MODEL ARCHITECTURE
class DQN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(8,128), nn.ReLU(),
            nn.Linear(128,128), nn.ReLU(),
            nn.Linear(128,3)
        )
    def forward(self, x):
        return self.net(x)


# LOAD TRAINED MODEL
def load_model(model_path="frequency_idqn_model.pth"):
    model = DQN()
    model.load_state_dict(torch.load(model_path, map_location="cpu"))
    model.eval()
    return model

model = load_model()


# FREQUENCY OPTIONS
FREQUENCIES = {
    0: 3.32,
    1: 3.34,
    2: 3.90
}


# RUN A PREDICTION
def pick_frequency(telemetry):
    """
    telemetry must contain:
      snr1, snr2, snr3, snr4,
      distance, lat, lon, alt
    """

    state = np.array([
        telemetry["snr1"],
        telemetry["snr2"],
        telemetry["snr3"],
        telemetry["snr4"],
        telemetry["distance"],
        telemetry["lat"],
        telemetry["lon"],
        telemetry["alt"]
    ], dtype=np.float32)

    x = torch.tensor(state).unsqueeze(0)
    qvals = model(x)
    action = torch.argmax(qvals, dim=1).item()

    return FREQUENCIES[action]
